In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from load_data import Data
from models import LogisticRegressor, Model, Analytics, LinearCombo
from sklearn.linear_model import LogisticRegression
import os
import pickle


## **Load Data**

In [30]:
data = Data()

pc_path = r"D:\nnUNetFrame\BaseLearnerInference"

training_images = [f"{a:03d}" for a in range(1, 33)]
all_images = [f"{a:03d}" for a in range(1, 41)]
testing_images = [f"{a:03d}" for a in range(33, 41)]

models = ["BasicPlans", "DA5_Segmentations", "LargeEncoder"]

seg_path = os.path.join(pc_path, "mySegmentations")
gt_path = os.path.join(pc_path, "GroundTruths")

data.get_simple_segmentations(seg_path, all_images, models)
data.get_groundtruths(gt_path)



In [31]:
model_segs = {model: {} for model in models}
for model in models:
    for image in testing_images:

        model_segs[model][image] = data.simple_data[image][model]



In [32]:
for model in models:
    model_analytics = Analytics(model_segs[model], data.gts)
    mean_dice = np.mean(model_analytics.dice_coefficients())
    print(f"{model}: {mean_dice}")


BasicPlans: 0.9747786337498603
DA5_Segmentations: 0.9744126209314119
LargeEncoder: 0.9743305622274461


## **Logistic Regression**

In [39]:
logreg = LogisticRegressor(data.simple_data, data.gts, models)

In [ ]:


def test_alpha(alpha):
    log_params = {'C': 0.1}
    dice = []
    for i in range(1, 33):
        train_fold = [k for k in training_images if k != f"{i:03d}"]
        logreg.train_model(train_fold, log_params)
        result = logreg.predict([f"{i:03d}"])
        analytics = Analytics(result, data.gts)
        dice.append(analytics.dice_coefficients()[0])
        
    with open(f'dice_results_C_{alpha:.6f}.pkl', 'wb') as f:
        pickle.dump({'C': alpha, 'dice_scores': dice, 'mean_dice': np.mean(dice)}, f)
    
    return dice


In [54]:
dices = {}
for a in np.linspace(-3, 3, 31):
    alpha = 10 ** a
    dice = test_alpha(alpha)
    dices[a] = dice


AttributeError: 'list' object has no attribute 'values'

In [41]:
logreg.train_model(training_images, log_params)

LogisticRegression(class_weight='balanced')

In [42]:
logistic_results = logreg.predict(testing_images)

In [43]:
logreg_analytics = Analytics(logistic_results, data.gts)

In [44]:
np.mean(logreg_analytics.dice_coefficients())

np.float64(0.9750212370972074)

In [3]:
lin = LinearCombo(data.simple_data, data.gts, models)

In [5]:
lin_preds = lin.predict_linear(training_images)

In [6]:
lin_anal = Analytics(lin_preds, data.gts)

In [7]:
thres = lin_anal.threshold()

In [9]:
l2 = Analytics(thres, data.gts)


In [11]:
np.mean(l2.dice_coefficients())

np.float64(0.9693246872496355)